# variables

In [ ]:
API_URL = "http://172.25.120.43.nip.io:8006"
API_KEY = "daky_jW8wqbpGaI"
API_SECRET = "8KNA97ExdcTo3ZFDc9S5sWXvbPibK0"
SOLVER_NAME = "simcore/services/comp/itis/sleeper"
SOLVER_VERSION = "2.1.6"

In [ ]:
import os
os.environ["OSPARC_DEV_FEATURES_ENABLED"] = "1"

# initialize connection

In [ ]:
from multiprocessing.pool import AsyncResult
from tenacity import AsyncRetrying, TryAgain, retry_if_exception_type
from tenacity.wait import wait_fixed
from tqdm.notebook import tqdm
from rich import print
import asyncio
import functools
import osparc
import typing


cfg = osparc.Configuration(
    host=f"{API_URL}",
    username=API_KEY,
    password=API_SECRET,
)
# cfg.debug = True

In [ ]:
with osparc.ApiClient(cfg) as api_client:
    profile = osparc.UsersApi(api_client).get_my_profile()
    print(profile)
    solvers_api = osparc.SolversApi(api_client)


# get solver

In [ ]:
solver = typing.cast(osparc.Solver, solvers_api.get_solver_release(SOLVER_NAME, SOLVER_VERSION))
print(solver)

# create job

In [ ]:
async def create_job(solver: osparc.Solver) -> osparc.Job:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.create_job,
            solver.id,
            solver.version,
            osparc.JobInputs(
                {
                    "input_2": 60,
                    "input_4": 0
                }
            ), async_req=True
        ))
    assert isinstance(result, AsyncResult) # nosec
    # print(job)
    return typing.cast(osparc.Job, await asyncio.get_running_loop().run_in_executor(None, result.get))

# list jobs

In [ ]:

def _list_jobs(solver: osparc.Solver) -> list[osparc.Job]:
    assert solver.id
    assert solver.version
    gen = solvers_api.jobs(solver.id, solver.version)
    return [job for job in gen if isinstance(job, osparc.Job)]

async def list_jobs(solver: osparc.Solver) -> list[osparc.Job]:
    return await asyncio.get_running_loop().run_in_executor(None, _list_jobs, solver)
    
jobs =await list_jobs(solver)
# print(jobs)
# assert len(jobs) == 0, f"found {len(jobs)} jobs"

# inspect job

In [ ]:
async def inspect_job(solver: osparc.Solver, job: osparc.Job)-> osparc.JobStatus:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.inspect_job, solver.id, solver.version, job.id,async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    # print(status)
    return typing.cast(osparc.JobStatus, await asyncio.get_running_loop().run_in_executor(None, result.get))

## get job result

In [ ]:
async def get_job_result(solver: osparc.Solver, job: osparc.Job) -> osparc.JobOutputs:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.get_job_outputs, solver.id, solver.version, job.id, async_req=True))
    assert isinstance(result, AsyncResult) # nosecregistry.staging.osparc.io/simcore/services/comp/itis/sleeper:2.0.2
    return typing.cast(osparc.JobOutputs, await asyncio.get_running_loop().run_in_executor(None, result.get))

# start job

In [ ]:
async def start_job(solver: osparc.Solver, job: osparc.Job) -> osparc.Job:
    result= await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.start_job, solver.id, solver.version, job.id,async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    return typing.cast(osparc.Job, await asyncio.get_running_loop().run_in_executor(None, result.get))

# set job metadata

In [ ]:
from typing import Any

async def set_job_metadata(solver: osparc.Solver, job: osparc.Job, job_metadata: dict[str, Any]) -> dict[str, Any]:
    result= await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.replace_job_custom_metadata, solver.id, solver.version, job.id, {"metadata":job_metadata},async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    return typing.cast(dict[str, Any], await asyncio.get_running_loop().run_in_executor(None, result.get))

# delete job

In [ ]:
async def delete_job(solver: osparc.Solver, job: osparc.Job) -> None:
    result = await asyncio.get_running_loop().run_in_executor(None, functools.partial(solvers_api.delete_job, solver.id, solver.version, job.id, async_req=True))
    assert isinstance(result, AsyncResult) # nosec
    await asyncio.get_running_loop().run_in_executor(None, result.get)

# run N sleepers

In [ ]:
NUM_JOBS = 2

jobs = []
with tqdm(total=NUM_JOBS, desc="Creating jobs") as pbar:
    for result in asyncio.as_completed([create_job(solver) for _ in range(NUM_JOBS)]):
        job = await result
        jobs.append(job)
        pbar.update()

In [ ]:
PARENT_NODE_ID = "7a7d6b38-7e9f-46b2-8cfb-4ee7ae20d0b1"
with tqdm(total=NUM_JOBS, desc="Setting jobs metadata") as pbar:
    for result in asyncio.as_completed([set_job_metadata(solver, job, job_metadata={"job_index": jobs.index(job), "node_id": PARENT_NODE_ID}) for job in jobs]):
        await result
        pbar.update()

In [ ]:
async def run_job(solver: osparc.Solver, job: osparc.Job) -> osparc.JobStatus:
    with tqdm(total=100) as pbar:
        await start_job(solver, job)

        job_status = await inspect_job(solver, job)
        assert job_status.progress is not None # nosec
        pbar.update(job_status.progress)
        current_progress = job_status.progress
        async for attempt in AsyncRetrying(wait=wait_fixed(1), retry=retry_if_exception_type()):
            with attempt:
                job_status = await inspect_job(solver, job)
                assert job_status.progress is not None # nosec
                if job_status.progress != current_progress:
                    pbar.update(job_status.progress - current_progress)
                    current_progress = job_status.progress
                    
                if not (job_status.progress == 100 and job_status.state in ["FAILED", "SUCCESS"]):
                    raise TryAgain
                
        if job_status.state == "FAILED":
            raise RuntimeError("job failed!")
        return job_status

job_statuses = []
with tqdm(total=len(jobs), desc="Running jobs") as pbar:
    for result in asyncio.as_completed([run_job(solver, job) for job in jobs]):
        try:
            job_statuses.append(await result)            
        except osparc.ApiException as exc:
            tqdm.write(f"Error while running {job.id}: {exc}")
        finally:
            pbar.update()

## Check results

In [ ]:
assert all(status.state == "SUCCESS" for status in job_statuses)

# Get job submission timestamps

In [ ]:
print(job_statuses[1])

## Get job result

In [ ]:
print(await get_job_result(solver, jobs[0]))

In [ ]:
jobs =await list_jobs(solver)
from tqdm.asyncio import tqdm_asyncio
await tqdm_asyncio.gather(*(delete_job(solver, job) for job in jobs))
jobs =await list_jobs(solver)
assert len(jobs) == 0, f"found {len(jobs)} jobs"